In this file I will test how well our current feature matrix predicts synergies using trees and random forests 

In [1]:
import pandas as pd
import numpy as np
import _pickle as pickle

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import MACCSkeys

#### Imports from Ammar's prettfied file

In [2]:
# Ammars output files
cellLine_info = pd.read_csv('output/cell_lines_enriched.csv', sep=',', dtype='string')
drug_info = pd.read_csv('output/all_drugs_with_smiles.csv', sep=',', dtype='string')

##### Ammar Celllines: apply One Hot Encoding on category and gender columns and clean the age column

In [3]:
cellLine_info.head()

,cellName,cosmicId,category,gender,age
0,786-0,905947,Cancer cell line,Male,58Y
1,A498,905948,Cancer cell line,Male,52Y
2,A549/ATCC,905949,Cancer cell line,Male,58Y
3,ACHN,905950,Cancer cell line,Male,22Y
4,CCRF-CEM,905952,Cancer cell line,Female,4Y


In [4]:
cat_ohe = pd.get_dummies(cellLine_info.category, prefix='cat')

In [5]:
cellLine_info_full = pd.concat([cellLine_info,cat_ohe], axis=1)

In [6]:
gender_ohe = pd.get_dummies(cellLine_info.gender, prefix='gender')

In [7]:
cellLine_info_full = pd.concat([cellLine_info_full,gender_ohe], axis=1)

In [8]:
cellLine_info_full['age'] = cellLine_info_full['age'].str.replace('Y','').replace('Age unspecified','0')
cellLine_info_full['age'] = cellLine_info_full['age'].astype('int')

In [9]:
cellLine_info_full['age'].unique()

array([58, 52, 22,  4, 70, 69,  0, 48, 36, 60, 62, 47, 53, 43, 19, 64, 51,
       42, 61, 24, 67, 34, 75, 11, 54, 31,  1, 33, 49, 74, 55, 46, 29, 50,
       56, 27, 18, 65, 59, 63, 39, 25])

In [10]:
cellLine_info_full = cellLine_info_full.drop('cosmicId', 1)
cellLine_info_full = cellLine_info_full.drop('category', 1)
cellLine_info_full = cellLine_info_full.drop('gender', 1)

In [11]:
cellLine_info_full.head()

,cellName,age,cat_Cancer cell line,cat_Embryonic stem cell,cat_Hybridoma,cat_Transformed cell line,gender_Female,gender_Male,gender_Sex unspecified
0,786-0,58,1,0,0,0,0,1,0
1,A498,52,1,0,0,0,0,1,0
2,A549/ATCC,58,1,0,0,0,0,1,0
3,ACHN,22,1,0,0,0,0,1,0
4,CCRF-CEM,4,1,0,0,0,1,0,0


#### cellline data demo

In [12]:
cellLine_info_full.dtypes

cellName                     string
age                           int64
cat_Cancer cell line          uint8
cat_Embryonic stem cell       uint8
cat_Hybridoma                 uint8
cat_Transformed cell line     uint8
gender_Female                 uint8
gender_Male                   uint8
gender_Sex unspecified        uint8
dtype: object

In [13]:
cellLine_info_full.head()

,cellName,age,cat_Cancer cell line,cat_Embryonic stem cell,cat_Hybridoma,cat_Transformed cell line,gender_Female,gender_Male,gender_Sex unspecified
0,786-0,58,1,0,0,0,0,1,0
1,A498,52,1,0,0,0,0,1,0
2,A549/ATCC,58,1,0,0,0,0,1,0
3,ACHN,22,1,0,0,0,0,1,0
4,CCRF-CEM,4,1,0,0,0,1,0,0


#### *DO NOT run block below unneccessarily!*

#### calculating descriptors

In [ ]:
drug_info.head()

,drugName,smilesString
0,Bendamustine,CN1C2=C(C=C(C=C2)N(CCCl)CCCl)N=C1CCCC(=O)O
1,Lonidamine,C1=CC=C2C(=C1)C(=NN2CC3=C(C=C(C=C3)Cl)Cl)C(=O)O
2,Lenalidomide,C1CC(=O)NC(=O)C1N2CC3=C(C2=O)C=CC=C3N
3,Cladribine,C1C(C(OC1N2C=NC3=C2N=C(N=C3N)Cl)CO)O
4,Pentostatin,C1C(C(OC1N2C=NC3=C2NC=NCC3O)CO)O


In [ ]:
# convert to molecules
%%capture --no-display
drug_mols = drug_info[["drugName","smilesString"]]
drug_mols.loc[:,'mols'] = [Chem.MolFromSmiles(mol) for mol in drug_mols.loc[:,"smilesString"]]
drug_mols.drop(["smilesString"],axis=1,inplace=True)
drug_mols.head(2)

,drugName,mols
0,Bendamustine,<rdkit.Chem.rdchem.Mol object at 0x7cf48be226c0>
1,Lonidamine,<rdkit.Chem.rdchem.Mol object at 0x7cf445b93d50>


In [ ]:
# prepare 2D descriptor calculator 
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

lipinskiDescriptors = ["FractionCSP3","HeavyAtomCount","NHOHCount","NOCount","NumAliphaticCarbocycles","NumAliphaticHeterocycles",
"NumAromaticCarbocycles","NumAromaticHeterocycles","NumHAcceptors","NumHDonors",
"NumHeteroatoms","NumRotatableBonds","NumSaturatedCarbocycles","NumSaturatedHeterocycles","RingCount"]

rdkitChemDescriptors = ["BalabanJ","BertzCT","Ipc","HallKierAlpha","Kappa1","Kappa2","Kappa3",
"Chi0","Chi1","Chi0n","Chi1n","Chi2n","Chi3n","Chi4n","Chi0v","Chi1v","Chi2v","Chi3v","Chi4v",
"MolLogP","MolMR","MolWt","HeavyAtomMolWt","NumValenceElectrons",
"NumAromaticRings","NumSaturatedRings","NumAliphaticRings","FractionCSP3","TPSA",
"LabuteASA"]

# I don't know about them and there are 57 of them. To see if they pop up as interesting I include 15 for now
MOEtypeDescriptors = ["PEOE_VSA1","PEOE_VSA2","PEOE_VSA3",
"SMR_VSA1","SMR_VSA2","SMR_VSA3",
"SlogP_VSA1","SlogP_VSA2","SlogP_VSA3",
"EState_VSA1","EState_VSA2","EState_VSA3",
"VSA_EState1","VSA_EState2","VSA_EState3"]

# descriptors not in: [Phi, NumAmideBonds, NumSpiroAtoms, NumBridgeheadAtoms, MQNs]
all2Ddescriptors = lipinskiDescriptors + rdkitChemDescriptors + MOEtypeDescriptors

molDescCalc = MolecularDescriptorCalculator(all2Ddescriptors)

# example mol
#molDescCalc.CalcDescriptors(mol=drug_mols.iloc[10,1])

In [ ]:
# 44 sec
# calculating all 2D descriptors
drug_feats = pd.DataFrame([molDescCalc.CalcDescriptors(mol) for mol in drug_mols.iloc[:,1]],columns=all2Ddescriptors)
drug_feats.set_index(drug_mols["drugName"], inplace=True)
drug_feats.head(2)

,FractionCSP3,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAromaticCarbocycles,NumAromaticHeterocycles,NumHAcceptors,NumHDonors,...,SMR_VSA3,SlogP_VSA1,SlogP_VSA2,SlogP_VSA3,EState_VSA1,EState_VSA2,EState_VSA3,VSA_EState1,VSA_EState2,VSA_EState3
drugName,,,,,,,,,,,,,,,,,,,,,
Bendamustine,0.500000,23,1,5,0,0,1,1,4,1,...,9.551078,4.89991,45.476431,18.263031,5.969305,6.420822,24.601650,2.022224,17.421914,8.744802
Lonidamine,0.066667,21,1,4,0,0,2,1,3,1,...,9.780485,0.00000,20.856317,6.544756,5.969305,5.693928,21.976247,1.632842,11.272932,15.096254


In [ ]:
%%capture --no-display
# 12 min
# making embeddings
from rdkit.Chem import AllChem

drug_mols["molEmbeds"] = pd.DataFrame([Chem.AddHs(mol) for mol in drug_mols.iloc[:,1]])

for i in range(len(drug_mols["molEmbeds"])):
    AllChem.EmbedMolecule(drug_mols.loc[i,"molEmbeds"])

RDKit ERROR: [16:27:25] UFFTYPER: Unrecognized charge state for atom: 41
RDKit ERROR: [16:27:29] UFFTYPER: Unrecognized charge state for atom: 11
RDKit ERROR: [16:27:29] UFFTYPER: Unrecognized atom type: Au5 (0)
RDKit ERROR: [16:29:17] UFFTYPER: Unrecognized atom type: Pt5 (1)
RDKit ERROR: [16:29:17] UFFTYPER: Unrecognized atom type: Pt5 (0)
RDKit ERROR: [16:29:17] UFFTYPER: Unrecognized atom type: Pt5 (1)
RDKit ERROR: [16:29:17] UFFTYPER: Unrecognized atom type: Pt5 (0)
RDKit ERROR: [16:29:17] UFFTYPER: Unrecognized atom type: Pt3 (0)
RDKit WARNING: [16:29:50] UFFTYPER: Warning: hybridization set to SP3 for atom 0
RDKit ERROR: [16:29:50] UFFTYPER: Unrecognized charge state for atom: 0
RDKit ERROR: [16:30:05] UFFTYPER: Unrecognized charge state for atom: 92
RDKit ERROR: [16:30:36] UFFTYPER: Unrecognized atom type: Ca (0)
RDKit ERROR: [16:30:36] UFFTYPER: Unrecognized atom type: Ca (0)
RDKit ERROR: [16:31:35] UFFTYPER: Unrecognized atom type: Co2 (0)
RDKit ERROR: [16:31:39] UFFTYPER: Un

In [ ]:
%%capture --no-display
from rdkit.Chem.Descriptors3D import *

# properties I want to find
funcLst = [Eccentricity,InertialShapeFactor,NPR1,NPR2,PMI1,PMI2,PMI3,RadiusOfGyration,SpherocityIndex]
strLst = ["Eccentricity","InertialShapeFactor","NPR1","NPR2","PMI1","PMI2","PMI3","RadiusOfGyration","SpherocityIndex"]


descr3Ds = pd.DataFrame(drug_mols["drugName"])
descr3Ds.reset_index(inplace=True)

for j in range(len(funcLst)):
    tempLst = []
    for i in range(len(drug_mols.iloc[:,2])):
        try:
            tempLst.append(funcLst[j](drug_mols.iloc[i,2]))
        except:
            tempLst.append(None)
    descr3Ds[strLst[j]] = tempLst
descr3Ds.head(2)

,index,drugName,Eccentricity,InertialShapeFactor,NPR1,NPR2,PMI1,PMI2,PMI3,RadiusOfGyration,SpherocityIndex
0,0,Bendamustine,0.993094,0.000967,0.117318,0.929722,960.955534,7615.37766,8191.025716,4.837408,0.107492
1,1,Lonidamine,0.971951,0.000981,0.235183,0.912555,930.166324,3609.23162,3955.082929,3.636559,0.280885


In [ ]:
allFeats = descr3Ds.join(drug_feats,on="drugName",how="left").drop_duplicates()
allFeats.drop("index",axis=1,inplace=True)
allFeats.head(2)

,drugName,Eccentricity,InertialShapeFactor,NPR1,NPR2,PMI1,PMI2,PMI3,RadiusOfGyration,SpherocityIndex,...,SMR_VSA3,SlogP_VSA1,SlogP_VSA2,SlogP_VSA3,EState_VSA1,EState_VSA2,EState_VSA3,VSA_EState1,VSA_EState2,VSA_EState3
0,Bendamustine,0.993094,0.000967,0.117318,0.929722,960.955534,7615.37766,8191.025716,4.837408,0.107492,...,9.551078,4.89991,45.476431,18.263031,5.969305,6.420822,24.601650,2.022224,17.421914,8.744802
1,Lonidamine,0.971951,0.000981,0.235183,0.912555,930.166324,3609.23162,3955.082929,3.636559,0.280885,...,9.780485,0.00000,20.856317,6.544756,5.969305,5.693928,21.976247,1.632842,11.272932,15.096254


In [ ]:
import _pickle as pickle

with open('drug_features.pkl', 'wb') as outp:
    pickle.dump(allFeats, outp)

#### Joining

In [14]:
with open('drug_features.pkl', 'rb') as inp:
    drug_feats = pickle.load(inp)

In [15]:
# drugCombDB data
data = pd.read_csv("../drugCombDBdata/synergy-score-data/REGRdrugcombs_scored.csv")
print(str(len(data))+" different Combinations")
data["Drug1"] = data["Drug1"].str.lower()
data["Drug2"] = data["Drug2"].str.lower()
data.head(2)

498865 different Combinations


,ID,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,HSA
0,1,5-fu,abt-888,A2058,1.72,6.26,-2.75,5.54
1,2,5-fu,abt-888,A2058,5.88,12.33,3.33,11.61


In [16]:
# double Data
dataReversed = data
dataReversed = dataReversed.rename(columns={"Drug1":"Drug2","Drug2":"Drug1"})

dataDouble = data.append(dataReversed)
dataDouble.drop("ID",axis=1,inplace=True)
dataDouble.reset_index(inplace=True)
dataDouble.drop("index",axis=1,inplace=True)
print(len(dataDouble)," records in this frame")
dataDouble.head(2)

997730  records in this frame


,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,HSA
0,5-fu,abt-888,A2058,1.72,6.26,-2.75,5.54
1,5-fu,abt-888,A2058,5.88,12.33,3.33,11.61


In [17]:
drug_feats.set_index("drugName",inplace=True)
drug_feats.index = drug_feats.index.str.lower()

In [18]:
d1_drugs = drug_feats.add_prefix("D1 ")
d2_drugs = drug_feats.add_prefix("D2 ")

dataDouble_drug1_feats = dataDouble.join(d1_drugs,on="Drug1",how="left")
dataDouble_all_feats = dataDouble_drug1_feats.join(d2_drugs,on="Drug2",how="left")

In [19]:
dataDouble_all_feats["Cell line"]

0          A2058
1          A2058
2          A2058
3          A2058
4          A2058
           ...  
997725    DIPG25
997726    DIPG25
997727    DIPG25
997728    DIPG25
997729    DIPG25
Name: Cell line, Length: 997730, dtype: object

In [20]:
cellLine_info_full[cellLine_info["cellName"]=="DIPG25"]

,cellName,age,cat_Cancer cell line,cat_Embryonic stem cell,cat_Hybridoma,cat_Transformed cell line,gender_Female,gender_Male,gender_Sex unspecified


In [21]:
dataDouble_all_feats.iloc[997725,2]

'DIPG25'

In [22]:
delete = dataDouble_all_feats.join(cellLine_info_full.set_index("cellName"),on="Cell line", how="left")#.dropna()

In [24]:
with open('final-Frame.pkl', 'wb') as outp:
    pickle.dump(delete, outp)

#### For the evaluation metrics, we need a combination ID
This does not exist here, so I'll create one

In [2]:
with open('final-Frame.pkl', 'rb') as inp:
    data = pickle.load(inp)
data = data.dropna() #loose here 200k data
data.reset_index(inplace=True,drop=True)
print("Data has records ",len(data))
data.head(2)

Data has records  763412


,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,HSA,D1 Eccentricity,D1 InertialShapeFactor,D1 NPR1,...,D2 VSA_EState2,D2 VSA_EState3,age,cat_Cancer cell line,cat_Embryonic stem cell,cat_Hybridoma,cat_Transformed cell line,gender_Female,gender_Male,gender_Sex unspecified
0,5-fu,abt-888,A2058,1.72,6.26,-2.75,5.54,0.956402,0.004792,0.292055,...,19.238682,3.448989,43.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,5-fu,abt-888,A2058,5.88,12.33,3.33,11.61,0.956402,0.004792,0.292055,...,19.238682,3.448989,43.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


concatenate Drug1 and Drug2 to one cell   ---- does not work, because sometimes they are switchet up

In [3]:
# concatenate Drug1 and 2 alphabetically - TIME: 14 mins
sorted_conc_drugs=[]
for row in range(len(data)):
    d1 = data.iloc[row]["Drug1"]
    d2 = data.iloc[row]["Drug2"]
    if d1 < d2:
        sorted_conc_drugs.append(d1+d2+data.iloc[row]["Cell line"])
    else:
        sorted_conc_drugs.append(d2+d1+data.iloc[row]["Cell line"])
sorted_conc_drugs = pd.Series(sorted_conc_drugs)

In [4]:
# attach the joined names to the dataframe
numerator = pd.DataFrame(list(set(sorted_conc_drugs)))
numerator.reset_index(inplace=True)

data['drug_drug2'] = sorted_conc_drugs
data = data.join(numerator.set_index(0),on="drug_drug2",how="left")

In [5]:
#get the 2 new columns to the beginning
 
cols = data.columns.tolist()
cols = cols[-2:] + cols[:-2]
cols = cols[0:1] + cols[2:9] + cols[1:2] + cols[9:]
data = data[cols]
data.head()

,drug_drug2,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,HSA,index,D1 Eccentricity,...,D2 VSA_EState2,D2 VSA_EState3,age,cat_Cancer cell line,cat_Embryonic stem cell,cat_Hybridoma,cat_Transformed cell line,gender_Female,gender_Male,gender_Sex unspecified
0,5-fuabt-888A2058,5-fu,abt-888,A2058,1.72,6.26,-2.75,5.54,300073,0.956402,...,19.238682,3.448989,43.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,5-fuabt-888A2058,5-fu,abt-888,A2058,5.88,12.33,3.33,11.61,300073,0.956402,...,19.238682,3.448989,43.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5-fuabt-888A2058,5-fu,abt-888,A2058,3.59,11.66,2.65,10.94,300073,0.956402,...,19.238682,3.448989,43.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,5-fuabt-888A2058,5-fu,abt-888,A2058,-0.85,5.15,-3.86,4.43,300073,0.956402,...,19.238682,3.448989,43.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5-fuazd1775A2058,5-fu,azd1775,A2058,12.29,15.77,10.40,18.66,190819,0.956402,...,31.601287,14.099736,43.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


#### making predictions

In [ ]:
#with open('final-Frame.pkl', 'rb') as inp:
#    data = pickle.load(inp)
#data = data.dropna() #loose here 200k data
#data.reset_index(inplace=True,drop=True)
#print("Data has records ",len(data))
#data.head(2)

Data has records  763412


,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,HSA,D1 Eccentricity,D1 InertialShapeFactor,D1 NPR1,...,D2 VSA_EState2,D2 VSA_EState3,age,cat_Cancer cell line,cat_Embryonic stem cell,cat_Hybridoma,cat_Transformed cell line,gender_Female,gender_Male,gender_Sex unspecified
0,5-fu,abt-888,A2058,1.72,6.26,-2.75,5.54,0.956402,0.004792,0.292055,...,19.238682,3.448989,43.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,5-fu,abt-888,A2058,5.88,12.33,3.33,11.61,0.956402,0.004792,0.292055,...,19.238682,3.448989,43.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor 
from sklearn.model_selection import train_test_split

Y = data.iloc[:,7]
X = data.iloc[:,8:]
X.drop(["D1 Ipc","D2 Ipc"],axis=1,inplace=True)

with open('intermediate-pickles/Y.pkl', 'wb') as outp:
    pickle.dump(Y, outp)
with open('intermediate-pickles/X.pkl', 'wb') as outp:
    pickle.dump(X, outp)

savepoint start training from here

In [2]:
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor 
from sklearn.model_selection import train_test_split

with open('intermediate-pickles/Y.pkl', 'rb') as inp:
    Y = pickle.load(inp)
with open('intermediate-pickles/X.pkl', 'rb') as inp:
    X = pickle.load(inp)

X_train,x_test,Y_train,y_test = train_test_split(X,Y,test_size=0.3,stratify=Y) #consider stratifying on combination

In [3]:
obs = pd.DataFrame({"combination_idx":x_test.iloc[:,0],"y_test":y_test})

In [7]:
regressor = DecisionTreeRegressor()
regressor.fit(X_train.iloc[:,1:],Y_train)
preds=regressor.predict(x_test.iloc[:,1:])

In [28]:
with open('intermediate-pickles/tree-Predictions.pkl', 'wb') as outp:
    pickle.dump(preds, outp)
with open('intermediate-pickles/tree-Predictions-y-test.pkl', 'wb') as outp:
    pickle.dump(obs, outp)

#### Applying the scoring function

In [ ]:
# now: compare preds to y_test in their metric

In [2]:
with open('./intermediate-pickles/tree-Predictions-y-test.pkl', 'rb') as inp:
    obs = pickle.load(inp)
with open('./intermediate-pickles/tree-Predictions.pkl', 'rb') as inp:
    pred = pickle.load(inp)

In [3]:
obs["pred"] = pred
obs

,combination_idx,y_test,pred
219343,6497,-0.71,-6.370000
554195,1717,-1.53,-10.290000
10289,2366,-0.24,0.718571
554254,378,0.26,8.030000
117071,5304,-1.21,-0.850000
...,...,...,...
327528,2122,-4.59,0.420000
61961,1648,-3.89,1.225000
677966,6249,-0.62,5.675000
158316,3025,-3.63,0.720000


In [5]:
%%capture --no-display

weighted_corr=[]
weights=[]
for i in set(obs['combination_idx']):

    n = len(obs.loc[obs['combination_idx'] == i])
    
    if (np.var(obs["y_test"][obs['combination_idx'] == i]) == 0 and n > 1):
        weighted_corr.append(0)
        weights.append(np.sqrt(n-1))
    elif (n > 1):
        weighted_corr.append(np.sqrt(n-1) * np.corrcoef(obs.loc[obs['combination_idx'] == i]["y_test"], obs.loc[obs['combination_idx'] == i]["pred"])[0,1])
        weights.append(np.sqrt(n-1))

needed: a combination ID

then: for each combination (of 2 drugs), make pearson correlation of the two vectors (if they occur more than once in the test set)

multiply this correlation with sqrt(len-1)



In [6]:
df = pd.DataFrame({"corr":weighted_corr,"weights":weights}).dropna()
sum(df["corr"])/sum(df["weights"])

0.1387171835711428

In [7]:
arr = np.array(obs[["pred"]][obs["combination_idx"]==1717])-np.array(obs[["y_test"]][obs["combination_idx"]==1717])

In [8]:
np.sqrt(sum(arr**2))

array([37.68209492])

#### second training cycle: Forest

In [4]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split

with open('intermediate-pickles/Y.pkl', 'rb') as inp:
    Y = pickle.load(inp)
with open('intermediate-pickles/X.pkl', 'rb') as inp:
    X = pickle.load(inp)

X_train,x_test,Y_train,y_test = train_test_split(X,Y,test_size=0.3,stratify=Y)

In [7]:
regressor = RandomForestRegressor(max_depth=4, random_state=0)
regressor.fit(X_train,Y_train)

predsTrain=regressor.predict(X_train)
predsTest=regressor.predict(x_test)

KeyboardInterrupt: 